<a href="https://colab.research.google.com/github/marcosfranca91/IA-ProjetoAula04-ImersaoIA-Alura/blob/main/05_ProjetoAula_ImersaoAlura_BuscaSemanticaEmDocumentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Iniciar e importar bibliotecas

In [2]:
!pip install -q -U google-generativeai

import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata

API_KEY=userdata.get('secret_key')
genai.configure(api_key=API_KEY)

Exibir models de IA do tipo de Embeddings

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Gerando embed

In [4]:
#Exemplo de embedding
title = "A proxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A proxima geração de IA para desenvolvedores e Google Workspace"
"\n"
"Artigo Completo:\n"
"\n"
"Gemini API & Google AI Studio: Uma maneira acessivel de explorar e criar prototipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.050832573, -0.045979068, -0.021730404, 0.018187165, 0.060917858, -0.00397301, -0.026923843, -0.024506282, 0.060843028, 0.057315536, 0.012139791, 0.020727383, -0.049023647, -0.019864216, 0.008177625, -0.018251402, 0.019238722, -0.004915236, -0.033632826, 0.0013092121, 0.010360646, 0.001952694, -0.035639897, -0.079900675, -0.015591236, 0.022381095, 0.013440373, -0.03671789, -0.044627763, 0.034824885, -0.05278414, 0.048676778, -0.037388645, 0.019001883, -0.02834127, -0.039836675, -0.019951457, -0.04331637, 0.0013871023, -0.000174281, 0.0044607427, -0.088127196, -0.018334035, 0.014004841, -0.006753847, -0.026399735, 0.043591317, 0.049614936, 0.028473277, -0.053708427, 0.02774839, 0.01728693, 0.061085615, -0.042647533, -0.00075608713, -0.0074088452, 0.030675799, -0.032022476, 0.026557293, 0.0082471715, -0.0041376026, 0.026979811, -0.011544429, 0.06029074, 0.02742224, -0.05733566, -0.04420019, -0.0032732466, 0.009920128, 0.038438115, -0.00068542553, 0.0034154085, 0.062817045

Criando textos base

In [6]:
DOCUMENT1 = {
    "Titulo": "Operação do sistema de controle climatico",
    "Conteudo": "O Googlecar tem um sistema de controle climatico que permite ajustar a temperatura e o fluxo de ar no carro"
}

DOCUMENT2 = {
    "Titulo": "Touchscreen",
    "Conteudo": "O seu googlecar tem uma grande tela sensivel ao toque que fornece acesso a uma variedade de recursos."
}

DOCUMENT3 = {
    "Titulo": "Mudança de marchas",
    "Conteudo": "Seu googlecar tem uma transmissão automatica. Para trocar as marchas, basta mover a alavanca de cambio"
}

documents = [DOCUMENT1,DOCUMENT2,DOCUMENT3]

Criando DataFrame(google sheets/excel

In [9]:
df = pd.DataFrame(documents)
df.columns = ["Titulo","Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de controle climatico,O Googlecar tem um sistema de controle climati...
1,Touchscreen,O seu googlecar tem uma grande tela sensivel a...
2,Mudança de marchas,Seu googlecar tem uma transmissão automatica. ...


In [10]:
model = "models/embedding-001"

Criando Função

In [11]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

Utilizar função e adicionar ao dataframe

In [12]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climatico,O Googlecar tem um sistema de controle climati...,"[-0.007870706, -0.01954483, -0.06639469, 0.046..."
1,Touchscreen,O seu googlecar tem uma grande tela sensivel a...,"[0.017508306, -0.042287774, -0.019251728, -0.0..."
2,Mudança de marchas,Seu googlecar tem uma transmissão automatica. ...,"[-0.005648272, -0.04190383, -0.021225141, 0.03..."


Criar funcao que Buscar dados na base de conhecimento e retorne a resposta

In [13]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

Utilizar a função criada para fazer a busca nos documentos

In [14]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu googlecar tem uma transmissão automatica. Para trocar as marchas, basta mover a alavanca de cambio


Utilizar o gemini para ter uma resposta mais criativa e descontraida

In [15]:
prompt = f"Reescreva esse texto de uma forma mais descontraida, sem adicionar informações que não façam parte do texto: {trecho}"

generation_config = {
    "candidate_count": 1,
    "temperature": 0.5
}

model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)


response = model_2.generate_content(prompt)
print(response.text)

E aí, galera! Seu Googlecar é tipo um carro de corrida, mas sem o volante! Para dar um gás, é só mexer na alavanca de câmbio, tipo um DJ mudando de música.
